In [1]:
import matplotlib.pyplot as plt
import numpy as np

"""In order to provide a better presentation of the graphs we use the rcParams options shown below."""

import matplotlib
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['figure.figsize'] = (11.0, 8.0)

In [2]:
import matplotlib.pyplot as plt
from numpy import linalg as LA
import scipy.special

import time
from scipy.integrate import odeint
from scipy.special import zeta
from random import choices

from numpy import sin, cos, power


In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import numpy as np

from scipy.optimize import minimize
#from qutip import *
#from qutip.piqs import *

import matplotlib.animation as animation
from IPython.display import HTML
from IPython.core.display import Image, display

In [4]:
"""Our system of differential equations belongs to the complex space, that is why we implemented the code suggested in"""
"""https://stackoverflow.com/questions/19910189/scipy-odeint-with-complex-initial-values."""

import time
import numpy as np
from scipy.integrate import odeint
from scipy.special import zeta
from random import choices

def odeintz(func, z0, t, **kwargs):
   
    
    """An odeint-like function for complex valued differential equations.

    Inputs:
    ----------
      -func: function associated to dr/dt=f(x;t), where x is the set of parameters and variables to be determined
      -z0: 1d array with length N*(5N-1)/2
      -t: 1d array from t=0 to t=tf (parameter set by the user)
      - **kwargs: keyword arguments related with external functions to be used in odeint
    
    Return:
      -z: multivariable array with the solution of the differential equation associated with each variable"""

    # Disallow Jacobian-related arguments.
    _unsupported_odeint_args = ['Dfun', 'col_deriv', 'ml', 'mu']
    bad_args = [arg for arg in kwargs if arg in _unsupported_odeint_args]
    if len(bad_args) > 0:
        raise ValueError("The odeint argument %r is not supported by "
                         "odeintz." % (bad_args[0],))

    # Make sure z0 is a numpy array of type np.complex128.
    z0 = np.array(z0, dtype=np.complex128, ndmin=1)

    def realfunc(x, t, *args):
        z = x.view(np.complex128)
        dzdt = func(z, t, *args)
        # func might return a python list, so convert its return
        # value to an array with type np.complex128, and then return
        # a np.float64 view of that array.
        return np.asarray(dzdt, dtype=np.complex128).view(np.float64)

    result = odeint(realfunc, z0.view(np.float64), t, **kwargs)

    if kwargs.get('full_output', False):
        z = result[0].view(np.complex128)
        infodict = result[1]
        return z, infodict
    else:
        z = result.view(np.complex128)
        return z

## Euler-Maruyama

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import random, zeros

from numba import jit

dt = 0.0003 # Time step.
T = 400 # Total time.
n = int(T / dt) # Number of time steps.
steps=int(T/dt)

times = np.linspace(0., T, n)


J=1
N=400

refg     = 1 #I multiply by sqrt(N) for getting back the hamiltonian of the form g(aS_+a^{\dagger}S+) 
refdelta=  0
refkappa= 10
refgamma= 0
refomega = 0.962*(refg **2)/refkappa


    
Omega =refomega*J
g_coef=refg*J
Delta=refdelta*J
kappa=refkappa*J
Gamma_phi=refgamma*J
    

Gamma_du=0  #Fixed


Stochastic=1   #Is the process stochastic?
sqrt_N=np.sqrt(N)
sqrt_2phi=np.sqrt(2*Gamma_phi)

theta=np.pi/2   #Initial state
phi=0



#set the initial values due to the measurements in the state |\psi>=cos(\theta/2)|0>+sin(\theta/2)|1>

from scipy import stats
mk = np.array([-1,1])


xp= (0.5*(1-np.sin(theta)*np.cos(phi)),0.5*(1+np.sin(theta)*np.cos(phi)))
yp= (0.5*(1-np.sin(theta)*np.sin(phi)),0.5*(1+np.sin(theta)*np.sin(phi)))
zp = ((np.cos(theta/2))**2, (np.sin(theta/2))**2)


#xp=(0.5,0.5)
#yp=(0.5,0.5)
#zp=(1,0)



custmx = stats.rv_discrete(name='custmx', values=(mk, xp))
custmy = stats.rv_discrete(name='custmy', values=(mk, yp))
custmz = stats.rv_discrete(name='custmz', values=(mk, zp))




@jit

def trayectory(Stochastic,N=N):
    sqrtdt = np.sqrt(dt)*Stochastic
    
    Results =zeros((3*N+2, n))
    Results[3*N][0]  =np.random.normal(0, 0.5)
    Results[3*N+1][0]=np.random.normal(0, 0.5)
    
    for j in range(N):
        
        a=custmx.rvs()
        b=custmy.rvs()
        c=custmz.rvs()
        
        Results[j][0]    =a  
        Results[j+N][0]  =b
        Results[j+2*N][0]=c
        
    
    for t in range(n-1):
        
        Normal_VA_1=random.randn() #np.random.normal(0, 1)
        Normal_VA_2=random.randn() #np.random.normal(0, 1)
        
        Results[3*N][t+1]   = Results[3*N][t]  +dt*Delta*Results[3*N+1][t]+\
            -kappa*Results[3*N][t]*dt+np.sqrt(kappa/2)* sqrtdt * Normal_VA_1
        Results[3*N+1][t+1] = Results[3*N+1][t]-dt*Delta* Results[3*N][t]+\
            -kappa*Results[3*N+1][t]*dt+np.sqrt(kappa/2)*sqrtdt * Normal_VA_2
        
        Normal_VA=random.randn()   #np.random.normal(0, 1)
        
        
        for i in range(N): 
            
            Results[i][t+1]    = Results[i][t]+dt*(-2*g_coef/sqrt_N)*Results[3*N+1][t]*Results[i+2*N][t]-\
                dt*Gamma_phi*Results[i][t]-sqrt_2phi*Results[i+N][t]* sqrtdt * Normal_VA
            
            Results[i+N][t+1]  = Results[i+N][t]+dt*(-2*g_coef/sqrt_N)*Results[3*N][t]*Results[i+2*N][t]-dt*Omega*Results[i+2*N][t]+\
                -dt*Gamma_phi*Results[i+N][t]+sqrt_2phi*Results[i][t]* sqrtdt * Normal_VA
            
            Results[i+2*N][t+1]= Results[i+2*N][t]+dt*(2*g_coef/sqrt_N)*Results[3*N][t]*Results[i+N][t]+\
                dt*(2*g_coef/sqrt_N)*Results[3*N+1][t]*Results[i][t]+dt*Omega*Results[i+N][t]
            
            Results[3*N][t+1]   += -(g_coef/(2*sqrt_N))*Results[i+N][t]*dt
            Results[3*N+1][t+1] += -(g_coef/(2*sqrt_N))*Results[i][t]*dt
            
    return Results


In [7]:
from numpy import mean
from numpy import linalg,transpose, dot



from numba import jit
@jit

def summary_stat(Res,N=N,n=n):
    Results =zeros((14, n))
    count=0
    while count<n:
        a,b,c=Res[0:N, count:count+1],Res[N:2*N, count:count+1],Res[2*N:3*N, count:count+1]
        
        Results[0][count],Results[1][count],Results[2][count]=N*mean(a)/2,N*mean(b)/2,N*mean(c)/2
        Results[3][count]=0.25*(  (N*mean(a))**2- (linalg.norm(a,2))**2 )
        Results[4][count]=0.25*(  (N*mean(b))**2- (linalg.norm(b,2))**2 )
        Results[5][count]=0.25*(  (N*mean(c))**2- (linalg.norm(c,2))**2 )
        Results[6][count]=(N*mean(a)/2)**2
        Results[7][count]=(N*mean(b)/2)**2
        Results[8][count]=(N*mean(c)/2)**2
        
        alpha_x=mean(Res[3*N:3*N+1, count:count+1])
        alpha_y=mean(Res[3*N+1:3*N+2, count:count+1])
        
        alpha_cuad=alpha_x**2+alpha_y**2
        
        Results[9][count]=alpha_cuad  #np.power(alpha_x,2)+np.power(alpha_y,2)-0.5
        Results[10][count]=alpha_cuad**2-2*alpha_cuad+0.5

        Results[11][count]=Results[0][count]*Results[1][count]-0.25*dot(transpose(a)[0],transpose(b)[0])    #xy
        Results[12][count]=Results[1][count]*Results[2][count]-0.25*dot(transpose(b)[0],transpose(c)[0])    #yz
        Results[13][count]=Results[2][count]*Results[0][count]-0.25*dot(transpose(c)[0],transpose(a)[0])    #zx
        
        count=count+1
        
    
    return Results   

In [8]:
from numpy import add
from numba import jit



itera=5000
@jit

def averages(Stochastic,itera=itera,N=N,n=n):
    Results =zeros((14, n))
    for j in range(itera):
        Trayec=trayectory(Stochastic,N=N)
        Results=add(Results,summary_stat(Trayec,N,n))
    
    return Results/itera
    
    
    
    

In [ ]:
t0=time.time()

itera=120

Result_varDDTWA_EM=averages(1,itera=itera,N=N,n=n)
#Result_varDTWA=averages(0,itera=itera,N=N,n=n)

t1=time.time()

print(t1-t0)



<ipython-input-8-6d45fc1f6958>:7: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "averages" failed type inference due to: Invalid use of type(CPUDispatcher(<function trayectory at 0x7f229c9c08c0>)) with parameters (int64, N=int64)

During: resolving callee type: type(CPUDispatcher(<function trayectory at 0x7f229c9c08c0>))
During: typing of call at <ipython-input-8-6d45fc1f6958> (12)


File "<ipython-input-8-6d45fc1f6958>", line 12:
def averages(Stochastic,itera=itera,N=N,n=n):
    <source elided>
    for j in range(itera):
        Trayec=trayectory(Stochastic,N=N)
        ^

  @jit
<ipython-input-8-6d45fc1f6958>:7: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "averages" failed type inference due to: cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-8-6d45fc1f6958>", line 11:
def averages(Stochastic,itera=itera,N=N,n=n):
    <source

## Results DDTWA

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics Photon Number', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle a^{\dagger}a \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[9]-0.5,"--",label="$a^{\dagger}a$ DDTWA EM")
#plt.plot(times, Result_varDTWA[9]-0.5,"--",label="$a^{\dagger}a$ DTWA EM")

#plt.plot(times, Result_varDDTWA_SH[9]-0.5,"--",label="$a^{\dagger}a$ DDTWA SH")


#plt.legend(fontsize = label_size)
plt.legend()
plt.show()
plt.close()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
plt.title(r'Quantum Evolution of $S_k$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle S_k \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[0],"--",label="$S_x$ DDTWA EM")

plt.plot(times, Result_varDDTWA_EM[1],"--",label="$S_y$ DDTWA EM")

plt.plot(times, Result_varDDTWA_EM[2],"--",label="$S_z$ DDTWA EM")

#plt.plot(times, Result_varDTWA[0],"-",label="$S_x$ DTWA EM")

#plt.plot(times, Result_varDTWA[1],"-",label="$S_y$ DTWA EM")

#plt.plot(times, Result_varDTWA[2],"-",label="$S_z$ DTWA EM")

#plt.plot(times, Result_varDDTWA_SH[0],"--",label="$S_x$ DDTWA SH")

#plt.plot(times, Result_varDDTWA_SH[1],"--",label="$S_y$ DDTWA SH")

#plt.plot(times, Result_varDDTWA_SH[2],"--",label="$S_z$ DDTWA SH")


plt.plot(times, 0*times,"-",label="$<S_k>=0$")

#plt.legend(fontsize = label_size)
plt.legend(loc="upper right")
plt.show()
plt.close()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics $a^{\dagger}a^{\dagger}aa$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle a^{\dagger}a^{\dagger} aa \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[10],"--",label="$a^{\dagger}a^{\dagger}aa$ DDTWA EM")
#plt.plot(times, Result_varDTWA[10],"--",label="$a^{\dagger}a^{\dagger}aa$ DTWA EM")
#plt.plot(times, Result_varDDTWA_SH[10],"--",label="$a^{\dagger}a^{\dagger}aa$ DDTWA SH")




#plt.legend(fontsize = label_size)
plt.legend()
plt.show()
plt.close()

In [ ]:
# DESCRIPTION OF MOMENTS OF S_k

Sxx=N/4+  Result_varDDTWA_EM[3+0]
Syy=N/4+  Result_varDDTWA_EM[3+1]
Szz=N/4+  Result_varDDTWA_EM[3+2]

#Sxx_nn=N/4+  Result_varDTWA[3+0]
#Syy_nn=N/4+  Result_varDTWA[3+1]
#Szz_nn=N/4+  Result_varDTWA[3+2]




In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics $\Delta (S_k^2)$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\Delta (S_k^2) (t)$', fontsize = label_size)


axEM=N/4+  Result_varDDTWA_EM[3+0]-(Result_varDDTWA_EM[0])**2
ayEM=N/4+  Result_varDDTWA_EM[3+1]-(Result_varDDTWA_EM[1])**2
azEM=N/4+  Result_varDDTWA_EM[3+2]-(Result_varDDTWA_EM[2])**2


#axEM_nn=N/4+  Result_varDTWA[3+0]-(Result_varDTWA[0])**2
#ayEM_nn=N/4+  Result_varDTWA[3+1]-(Result_varDTWA[1])**2
#azEM_nn=N/4+  Result_varDTWA[3+2]-(Result_varDTWA[2])**2


#Use for unitary dynamics solely

#ax1EM=np.array(Result_varDDTWA_EM[6+0])-np.array((Result_varDDTWA_EM[0])**2)
#ay1EM=np.array(Result_varDDTWA_EM[6+1])-np.array((Result_varDDTWA_EM[1])**2)
#az1EM=np.array(Result_varDDTWA_EM[6+2])-np.array((Result_varDDTWA_EM[2])**2)




#axSH=N/4+  Result_varDDTWA_SH[3+0]-(Result_varDDTWA_SH[0])**2
#aySH=N/4+  Result_varDDTWA_SH[3+1]-(Result_varDDTWA_SH[1])**2
#azSH=N/4+  Result_varDDTWA_SH[3+2]-(Result_varDDTWA_SH[2])**2


#Use for unitary dynamics solely

#ax1SH=np.array(Result_varDDTWA_SH[6+0])-np.array((Result_varDDTWA_SH[0])**2)
#ay1SH=np.array(Result_varDDTWA_SH[6+1])-np.array((Result_varDDTWA_SH[1])**2)
#az1SH=np.array(Result_varDDTWA_SH[6+2])-np.array((Result_varDDTWA_SH[2])**2)



plt.plot(times,axEM,"-.",label="DDTWA $(\Delta S_x)^2 $ EM")
plt.plot(times,ayEM,"-.",label="DDTWA $(\Delta S_y)^2 $ EM")
plt.plot(times,azEM,"-.",label="DDTWA $(\Delta S_z)^2 $ EM")


#plt.plot(times,axEM_nn,"-",label="DTWA $(\Delta S_x)^2 $ EM")
#plt.plot(times,ayEM_nn,"-",label="DTWA $(\Delta S_y)^2 $ EM")
#plt.plot(times,azEM_nn,"-",label="DTWA $(\Delta S_z)^2 $ EM")
#plt.plot(times,axSH,"-.",label="DDTWA $(\Delta S_x)^2 $ SH")
#plt.plot(times,aySH,"-.",label="DDTWA $(\Delta S_y)^2 $ SH")
#plt.plot(times,azSH,"-.",label="DDTWA $(\Delta S_z)^2 $ SH")


plt.legend()

plt.legend(loc="upper right")

In [ ]:
f = lambda x, Sxx, Syy, Szz, Sxys, Syzs, Szxs, Sx, Sy, Sz: Sxx*power(cos(x[0])*sin(x[1]),2)+\
    Syy*power(sin(x[0])*sin(x[1]),2)+Szz*power(cos(x[1]),2)+Sxys*sin(2*x[0])*power(sin(x[1]),2)+\
    Syzs*sin(2*x[1])*sin(x[0])+Szxs*sin(2*x[1])*cos(x[0])


Squeez=[]

Arraytime=[]

separ=1000


for j in range(int(steps/separ)):
    
    j=separ*j
    
    Sx= Result_varDDTWA_EM[0][j]
    Sy= Result_varDDTWA_EM[1][j]
    Sz= Result_varDDTWA_EM[2][j]
    
    Sxx= N/4+Result_varDDTWA_EM[3][j]
    Syy= N/4+Result_varDDTWA_EM[4][j]
    Szz= N/4+Result_varDDTWA_EM[5][j]
    
    Sxy= Result_varDDTWA_EM[11][j]
    Syz= Result_varDDTWA_EM[12][j]
    Szx= Result_varDDTWA_EM[13][j]
    
    cons = ({'type': 'eq', 'fun': lambda x: Sx*cos(x[0])*sin(x[1])+Sy*sin(x[0])*sin(x[1])+Sz*cos(x[1])})
    
    res = minimize(f, [0,0], constraints=cons, args=(Sxx,Syy,Szz,Sxy,Syz,Szx,Sx,Sy,Sz), tol=1e-17)
    Squeez.append(res.fun*N/(Sx**2+Sy**2+Sz**2+1e-6))
    
    Arraytime.append(times[j])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'$\xi^{2}$ Dynamics', fontsize = label_size)

plt.plot(Arraytime, np.array(Squeez),label="Spin Squeezing");

plt.legend()

In [ ]:
from scipy import integrate

In [ ]:
def time_averaging(array,indexes):
    result=[]
    lower_index=int(0.5*len(array))
    
    
    for j in range(len(array)):
        if j<lower_index:
            x=times[j:j+indexes]
            y=array[j:j+indexes]
            y_aver=integrate.trapz(y, x)/(x[-1]-x[0])
            result.append(y_aver)
            
        else:
            x=times[j-indexes:j]
            y=array[j-indexes:j]
            y_aver=integrate.trapz(y, x)/(x[-1]-x[0])
            result.append(y_aver)
    
    return result

In [ ]:
Squeezing_aver=time_averaging( -10*np.log10(np.array(Squeez))  ,int(len(Arraytime)*0.05));



In [ ]:
plt.plot(Arraytime, Squeezing_aver,label="Spin Squeezing");


In [ ]:
Squeezing_aver[-int(0.015*len(Arraytime))]

In [ ]:
Sz_aver=time_averaging( Result_varDDTWA_EM[2],int(len(Result_varDDTWA_EM[2])*0.05));
plt.plot(times, Sz_aver,label="Sz");


In [ ]:
Sz_aver[-int(0.015*len(Result_varDDTWA_EM[2]))]

In [ ]:
len(Result_varDDTWA_EM[0])

In [ ]:
j=40000

Sx= Result_varDDTWA_EM[0][j]
Sy= Result_varDDTWA_EM[1][j]
Sz= Result_varDDTWA_EM[2][j]
    
Sxx= N/4+Result_varDDTWA_EM[3][j]
Syy= N/4+Result_varDDTWA_EM[4][j]
Szz= N/4+Result_varDDTWA_EM[5][j]
    
Sxy= Result_varDDTWA_EM[11][j]
Syz= Result_varDDTWA_EM[12][j]
Szx= Result_varDDTWA_EM[13][j]

print(Sx,Sy,Sz,Sxx,Syy,Szz,Sxy,Syz,Szx)

In [ ]:
len(Squeez)

In [ ]:
print(Squeez[3999],Squeez[4000],Squeez[4001])